# Parsing and generating robot descriptions from templates

## Parsing XACRO files

`.xacro` can also be parsed and inspected using the `pcg_gazebo_pkgs` library.

For this example please clone the `universal_robot` repository into you `catkin` workspace

```
cd $HOME/catkin_ws/src
git clone https://github.com/ros-industrial/universal_robot.git
```

and then build and source the catkin workspace.

In [1]:
import warnings
warnings.filterwarnings('ignore')

from pcg_gazebo.simulation import SimulationModel
from pcg_gazebo.parsers import parse_xacro

In [2]:
import os
import rospkg
try:
    pkg_path = rospkg.RosPack().get_path('ur_description')
    UR5_AVAILABLE = True
except rospkg.ResourceNotFound:
    UR5_AVAILABLE = False
    print('ur_description package not available')
    
if UR5_AVAILABLE:
    urdf_filename = os.path.join(pkg_path, 'urdf', 'ur5_robot.urdf.xacro')

Load the URDF model as a `SimulationModel`.

In [3]:
if UR5_AVAILABLE:
    model = SimulationModel.from_urdf(parse_xacro(urdf_filename))

line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
line 75.
line 80.


In [4]:
if UR5_AVAILABLE:
    print('Links: ', model.link_names)

Links:  ['wrist_1_link', 'shoulder_link', 'wrist_2_link', 'upper_arm_link', 'wrist_3_link', 'forearm_link', 'world']


In [5]:
if UR5_AVAILABLE:
    print('Joints: ', model.joint_names)

Joints:  ['shoulder_lift_joint', 'wrist_2_joint', 'shoulder_pan_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_3_joint']


In [6]:
if UR5_AVAILABLE:
    model.show(mesh_type='collision')

![sim_ur5_collision](images/sim_ur5_collision.png)

In [7]:
if UR5_AVAILABLE:
    model.show(mesh_type='visual')

![sim_ur5_visual](images/sim_ur5_visual.png)

## Parsing Jinja templates to generate SDF robot descriptions 

Jinja is a powerful templating engine for Python. It can be extended with new functions and offers data structures such as dictionaries to be used inside the template.

In this example, the **kobuki** model was rewritten as a Jinja template (see below) to generate an SDF model for the robot. 

The extensions for the Jinja engines in the `pcg_gazebo` include 

* path completion using `$(find pkg)`, `package://`, `file://` 
* path completion for relative paths using `{% 'relative/path'|find_file %}` 
* macro functions used, for example, to compute moments of inertia for geometric primitives

In [8]:
from pcg_gazebo.simulation import SimulationModel
from pcg_gazebo.parsers import parse_sdf
from pcg_gazebo.utils import process_jinja_template

Open and process the Jinja template.

The output XML is provided as a `string`.

In [9]:
import os
import rospkg
jinja_template = os.path.abspath(
    os.path.join('robot_description', 'kobuki', 'sdf', 'kobuki.sdf.jinja'))
print('Jinja template = {}'.format(jinja_template))
output_xml = process_jinja_template(
    jinja_template,
    parameters=dict(robot_namespace='kobuki'))

Jinja template = /data/pcg_gazebo-master/examples/robot_description/kobuki/sdf/kobuki.sdf.jinja


Generate the SDF object for the XML output.

In [10]:
sdf = parse_sdf(output_xml)
print(sdf.models)

Generate a model from the **kobuki** SDF data.

In [11]:
model = SimulationModel.from_sdf(sdf.models[0])

line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
line 75.
line 80.


Visualize the model's collision and visual geometries

In [12]:
model.show(mesh_type='visual')

SceneViewer(width=1800, height=1028)

In [13]:
model.show(mesh_type='collision')

SceneViewer(width=1800, height=1028)

In [14]:
from pcg_gazebo.generators.creators import create_models_from_config
from pcg_gazebo.task_manager import Server, get_rostopic_list

# Start an empty world Gazebo simulation
server = Server()
server.create_simulation('default')
simulation = server.get_simulation('default')
simulation.create_gazebo_empty_world_task()
simulation.run_task('gazebo')

# Create a Gazebo proxy
gazebo_proxy = simulation.get_gazebo_proxy()

2020-09-10 17:31:23,732 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-10 17:31:23,999 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-10 17:31:24,247 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-10 17:31:24,504 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1


Spawn the **kobuki** model.

In [15]:
model.spawn(gazebo_proxy=gazebo_proxy)

line 81.
line 82.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.


True

List all ROS topics

In [16]:
print('ROS topics: ', get_rostopic_list(gazebo_proxy.ros_config.ros_master_uri))

ROS topics:  ['/clock', '/gazebo/link_states', '/gazebo/model_states', '/gazebo/parameter_descriptions', '/gazebo/parameter_updates', '/gazebo/set_link_state', '/gazebo/set_model_state', '/gazebo_gui/parameter_descriptions', '/gazebo_gui/parameter_updates', '/kobuki/cmd_vel', '/kobuki/groundtruth', '/rosout', '/rosout_agg', '/tf']


In [17]:
simulation.init_task(
    name='command',
    command='rostopic pub /kobuki/cmd_vel geometry_msgs/Twist "{linear: {x: {vx}, y: 0.0, z: 0.0}, angular: {x: 0.0, y: 0.0, z: {vtheta}}}"',
    has_gazebo=False,
    params=dict(
        vx=0.3,
        vtheta=0.4))
simulation.run_task('command')

![sim_kobuki_cmd](images/sim_kobuki_cmd.png)

In [ ]:
simulation.kill_all_tasks()

2020-09-10 17:32:42,598 | WARNING | task | Task <roscore> - Killing process tree...
2020-09-10 17:32:42,615 | WARNING | task | Sending SIGINT to child process id=12407
2020-09-10 17:32:42,616 | WARNING | task | Child process 12407 still running
2020-09-10 17:32:42,616 | WARNING | task | Sending SIGINT to child process id=12422
2020-09-10 17:32:42,618 | WARNING | task | Child process 12422 still running
2020-09-10 17:32:42,619 | WARNING | task | Sending SIGINT to child process id=12438
2020-09-10 17:32:42,620 | WARNING | task | Child process 12438 still running
2020-09-10 17:32:42,621 | WARNING | task | Sending SIGINT to child process id=12406
2020-09-10 17:32:42,623 | WARNING | task | Child process 12406 still running
2020-09-10 17:32:42,763 | WARNING | task | Process 12422 already terminated
2020-09-10 17:32:42,937 | WARNING | task | Process 12407 already terminated
2020-09-10 17:32:42,938 | WARNING | task | Process 12438 already terminated
2020-09-10 17:32:42,939 | WARNING | task | P